In [1]:
import numpy as np
import pandas as pd


In [2]:
step = 0.01 # -- time step
mass = 1.0 #kg -- mass
g = 9.81 #m/s/s -- gravity
        
vo = 5.0 #m/s -- initial velocity
yo = 0.0 #m -- initial position
to = 0.0 #s -- initial time
        
density = 1000 #kg/m3 -- density of water
mu = 8.9 * 10**(-4) #Pa s -- viscosity (water at 25 deg celsius)
radius = 0.01 #m (1 cm) -- radius

In [3]:
# coeff
def b():
    return (6 * np.pi) * mu * radius

In [ ]:
# y'' = f(v) = acceleration (changed return to addition from subtraction)
def f(v):
    zo = b() * v / mass
    return (-g + zo)

In [ ]:
# velocity calculated with Euler
def euler_vel():
    time = [to]
    vel = [vo]

    for i in range(75):
        v = vel[i]
        a = f(v)
        vn = v + a * step
        if vn <=0:
            vel.append(0)
        else:
            vel.append(vn)

        time.append(step*float(i+1))
    return time, vel

In [ ]:
# position calculated with Euler
def euler_pos(v):
    pos = [yo]

    for i in range(75):
        y = pos[i]
        f = v[i]
        yn = y + f * step
        pos.append(yn)

    return pos

In [ ]:
# analytical position model 
def pos(t):
    position = []

    for i in range(len(t)):
        zo = (-5.0) * mass / b()
        z1 = -g * (mass**2) / (b()**2)

        e = np.exp(-b() * t[i] / mass)
        z2 = (5.0 * mass) / b()
        z3 = (g * mass**2) / (b()**2)
        z4 = (-g * mass * t[i]) / b()
        
        y = ((zo + z1) * e + z2 + z3 + z4)
        if y<=0:            
            position.append(0)
        else:
            position.append(y)

    return position

In [ ]:
# analytical velocity model
def vel(t):
    velocity = []

    for i in range(len(t)):
        zo = (-5.0) * mass / b()
        z1 = -g * (mass**2) / (b()**2)
        z2 = (-b()/mass)

        e = np.exp(-b() * t[i] / mass)
        z3 = -g * mass / b()
        
        v = (zo + z1) * z2 * e + z3
        if v<=0:
            velocity.append(0)
        else:
            velocity.append(v)

    return velocity

In [ ]:
# euler error eval
def euler_err():
    t = euler_vel()[0]
    vel = vel(t)
    pos = pos(t)

    v = euler_vel()[1]
    y = euler_pos(v)

    df = pd.DataFrame({'time': t, 'Velocity': v, 'Euler_vel': vel})
    e_y = []
    e_v = []

    for i in range(df.shape[0]):
        e_v.append(np.abs(df['Velocity'][i]-df['Euler_vel'][i]))

    df['Euler_vel_err'] = e_v
    df['Position'] = pos
    df['Euler_pos'] = y

    for i in range(df.shape[0]):
        e_y.append(np.abs(df['Position'][i]-df['Euler_pos'][i]))

    df['Euler_pos_err'] = e_y

    return df

In [ ]:
# quality control
def rk_vel():
    time = [to]
    vel = [vo]

    for i in range(75):
        v = vel[i]
        vn = v + f(v+step*f(v)) * step
        if vn<=0:
            vel.append(0)
        else:
            vel.append(vn)

    return vel

In [ ]:
# re-examine this function
def rk_pos(self, v):
    time = [self.to]
    pos = [self.yo]

    for i in range(75):
        y = pos[i]
        a = v[i]
        yn = y + a * self.step
        pos.append(yn)

    return pos

In [ ]:
#rk error eval
def rk_err(self):
    t = self.euler_vel()[0]
    vel = self.vel(t)
    pos = self.pos(t)

    v = self.rk_vel()
    y = self.rk_pos(v)

    df = pd.DataFrame({'time': t, 'Velocity': vel, 'rk_vel': v})
    e_y = []
    e_v = []

    for i in range(df.shape[0]):
        e_v.append(np.abs(df['Velocity'][i]-df['rk_vel'][i]))

    df['rk_vel_error'] = e_v
    df['Position'] = pos
    df['rk_pos'] = y

    for i in range(df.shape[0]):
        e_y.append(np.abs(df['Position'][i]-df['rk_pos'][i]))

    df['rk_pos_err'] = e_y

    return df

In [ ]:
if __name__== "__main__":
C = calc()
print(C.euler_err())
print(C.rk_err())